In [6]:
from transformers import BertTokenizer,BertForMaskedLM
import torch 
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")




Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


'paris'

In [104]:
text = "The capital of Iran is [MASK]."
text = "Yesterday, I played [MASK] in the stadium"
text = "Yesterday, I played [MASK] in the [MASK]."
text = "I'm studying [MASK] learning in my computer class."
text = "I'm a very [MASK] player in football."
inputs = tokenizer(text, return_tensors="pt")
print(tokenizer.mask_token_id)

103


In [105]:
print(inputs.input_ids)

tensor([[ 101, 1045, 1005, 1049, 1037, 2200,  103, 2447, 1999, 2374, 1012,  102]])


In [106]:
with torch.no_grad():
    logits = model(**inputs).logits

# retrieve index of [MASK]
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
print(inputs.input_ids == tokenizer.mask_token_id)
print(mask_token_index)


tensor([[False, False, False, False, False, False,  True, False, False, False,
         False, False]])
tensor([6])


In [145]:
import numpy as np
import editdistance
import pandas as pd

word = 'skiled'
token_ids = np.argpartition(logits[0, mask_token_index], -5, axis=-1)[0,-5:]
scores = logits[0, mask_token_index][0, token_ids]
df = pd.DataFrame({'token_id':token_ids, 'scores':scores})
df['word'] = df['token_id'].apply(lambda row: tokenizer.decode([row]))
df['edit_distance'] = df['word'].apply(lambda row: editdistance.eval(word,row))
df.head()
# print(df['edit_distance'].idxmin())


,token_id,scores,word,edit_distance
0,5281,9.283118,experienced,8
1,10571,9.760030,skilled,1
2,7070,9.974232,valuable,7
3,10904,11.609438,talented,6
4,2204,12.591966,good,5


In [112]:
import editdistance
editdistance.eval('abxx', 'ababx')

2